#  Neural Networks Learning

## Introduction

In this lab, you will implement the feedforward algorithm for neural networks and apply it to the task of `Fashion MNIST`
recognition.

The same multi-class classification problem solved in the previous lab will be solved using Neural Networks. The same dataset will be used.

Before we begin with the exercises, we need to import all libraries required for this programming exercise. Throughout the notebook, we will be using [`numpy`](http://www.numpy.org/) for all arrays and matrix operations, [`matplotlib`](https://matplotlib.org/) for plotting, and [`scipy`](https://docs.scipy.org/doc/scipy/reference/) for scientific and numerical computation functions and tools.

## Motivation.

In order to capture more complex hypothesis, addition of non-linear terms in Logistic Regression based Classification quickly falls short with data containing higher number of features, since the number of terms in the hypothesis will grow asymptotically with an order of **O(n<sup>2</sup>)**

In [1]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np
import pandas as pd
# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# will be used to load MATLAB mat datafile format
from scipy.io import loadmat

# tells matplotlib to embed plots within the notebook
%matplotlib inline

In [13]:
def displayData(X, example_width=None):
    """
    Displays 2D data stored in X in a nice grid.
    """
    
    # selects a random set of rows from X
    rand_indices = np.random.choice(X.shape[0], 100, replace=False)

    # selects the rows of X using the random indices
    sel = X[rand_indices, :]

    # gets the number of rows and columns in the dataset
    m, n = sel.shape

    # sets example_width automatically if not passed in
    if example_width is None:
        example_width = int(np.round(np.sqrt(n)))

    for i in range(m):
        # gets the i-th row of X
        row = sel[i, :]

        # reshape the row into 20x20 pixel matrix
        row = row.reshape(example_width, example_width, order='F')

        # displays the 20x20 pixel matrix as a gray scale image
        pyplot.imshow(row, cmap=pyplot.cm.gray, interpolation='nearest')

        # don't show axis
        pyplot.axis('off')

    # displays the plot
    pyplot.show()

def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z+1e-10))

def tanh(z):
    return np.tanh(z)

## Neural Networks

In this exercise, you will implement the feedforward propagation for neural networks to predict handwritten digits with the optimized weights already provided. Later, the backpropagation algorithm will also be implemented to learn the parameters for the neural network.

We start the exercise by first loading the dataset. 

In [14]:
#  training data stored in arrays X, y
data = pd.read_csv(r'C:\Users\shirs\Desktop\APL745 Deep Learning\Lab 3\fashion-mnist_train.csv')

X ,y = data.iloc[:, 1:], data.iloc[:, 0].ravel()


### 1.1 Visualizing the data

There are 10000 training examples in `fashion-mnist_train`, where each training example is a 28 pixel by 28 pixel grayscale image of the digit. Each pixel is represented by a floating point number indicating the grayscale intensity at that location. The 28 by 28 grid of pixels is “unrolled” into a 784-dimensional vector. Each
of these training examples becomes a single row in our data matrix $X$. This gives us a 10000 by 784 matrix $X$ where every row is a training example for a handwritten digit image.

$$ X = \begin{bmatrix} - \left(x^{(1)} \right)^T - \\
- \left(x^{(2)} \right)^T - \\
\vdots \\
- \left(x^{(m)} \right)^T - \\
\end{bmatrix}
$$

The second part of the training set is a 10000-dimensional vector `y` that contains labels for the training set. 
The following cell randomly selects 100 images from the dataset and plots them.

In [16]:
# Randomly select 100 data points to display
rand_indices = np.random.choice(X.shape[0], 100)
sel = X.iloc[rand_indices, :]
displayData(sel,  example_width=28)

TypeError: '(array([33, 51, 38, 14, 41, 71, 89,  1, 64, 72, 46, 34, 60, 30, 43, 81, 74,
       54, 50, 77,  4, 98, 15, 11, 73, 29, 13, 27, 88, 86, 63, 78, 95,  8,
       82, 90, 93,  5,  2, 21, 83, 24, 52, 75,  6, 35, 32, 87, 28, 45, 94,
       65, 36, 76, 61, 69, 20, 85, 17, 97, 91, 16, 67, 39, 31,  9, 84, 25,
       22, 18, 92, 62, 58, 53,  3, 68, 42, 99, 44, 70, 48, 47,  0, 55, 40,
        7, 12, 80, 96, 23, 49, 57, 26, 37, 66, 59, 19, 56, 79, 10]), slice(None, None, None))' is an invalid key

### 1.2 Model representation



Our neural network is shown in the following figure.

![](Figures/neural_network.png)

It has 3 layers - an input layer, a hidden layer and an output layer. Recall that our inputs are pixel values
of digit images. Since the images are of size $28 \times 28$, this gives us 784 nput layer units (not counting the extra bias unit which always outputs +1). The training data was loaded into the variables `X` and `y` above.


In [17]:
# Setup the parameters you will use for this exercise
input_layer_size = 28*28   # 28x28 Input Images of Digits
hidden_layer_size = 25 # 25 hidden units
num_labels = 10          # 10 labels, from 0 to 9

# Initialise the random weights into variables w1 and w2

w1 = np.random.rand(hidden_layer_size, input_layer_size)
w2 = np.random.rand(num_labels, hidden_layer_size)


# w1 has size 
# w2 has size 
print('w1 has size ', w1.shape)
print('w2 has size ', w2.shape)




# Unroll parameters 
nn_params = np.concatenate((w1.ravel(), w2.ravel()), axis=0)

print(nn_params)

w1 has size  (25, 784)
w2 has size  (10, 25)
[0.14608374 0.88837102 0.50975937 ... 0.08232788 0.76815713 0.70253861]


In [18]:
# j --->j+1 sj, s(j+1); s(j+1)*s(j)+1
w1.shape, w2.shape[0]

((25, 784), 10)

### 1.3 Feedforward Propagation and Prediction

Now you will implement feedforward propagation for the neural network. You will need to complete the code in the function
`predict` to return the neural network’s prediction. You should implement the feedforward computation that computes
$h_\theta(x^{(i)})$ for every example $i$ and returns the associated predictions. Similar to the one-vs-all classification
strategy, the prediction from the neural network will be the label that has the largest output $\left( h_\theta(x) \right)_k$.

In [19]:
def predict(w1, w2, X):
    """
    Predict the label of an input given a trained neural network.
    
    Parameters
    ----------
    w1 : array_like
        Weights for the first layer in the neural network.
        It has shape (2nd hidden layer size x input size)
    
    w2: array_like
        Weights for the second layer in the neural network. 
        It has shape (output layer size x 2nd hidden layer size)
    
    X : array_like
        The image inputs having shape (number of examples x image dimensions).
    
    Return 
    ------
    p : array_like
        Predictions vector containing the predicted label for each example.
        It has a length equal to the number of examples.
    

    """
    if X.ndim==1:
        X=X[None]
    
    m=X.shape[0]
    num_labels= w2.shape[0]
    
    p= np.zeros(m)

    X = np.concatenate((np.ones((m, 1)), X), axis=1)
    z2 = X.dot(w1.T)
    a2 = sigmoid(z2)
    a2 = np.concatenate((np.ones((m, 1)), a2), axis=1)
    z3 = a2.dot(w2.T)
    a3 = sigmoid(z3)
    p = np.argmax(a3, axis=1)


    return p

## Feedforward Method

### Unregularized Cost Function

Now you will implement the cost function and gradient for the neural network.

Recall that the cost function for the neural network (without regularization) is:

$$ J(w) = \frac{1}{m} \sum_{i=1}^{m}\sum_{k=1}^{K} \left[ - y_k^{(i)} \log \left( \left( h_w \left( x^{(i)} \right) \right)_k \right) - \left( 1 - y_k^{(i)} \right) \log \left( 1 - \left( h_w \left( x^{(i)} \right) \right)_k \right) \right]$$

where $h_w \left( x^{(i)} \right)$ is computed as shown in the neural network figure above, and K = 10 is the total number of possible labels. Note that $h_w(x^{(i)})_k = a_k^{(3)}$ is the activation (output
value) of the $k^{th}$ output unit. Also, recall that whereas the original labels (in the variable y) were 0, 1, ..., 9, for the purpose of training a neural network, we need to encode the labels as vectors containing only values 0 or 1, so that

$$ y = 
\begin{bmatrix} 1 \\ 0 \\ 0 \\\vdots \\ 0 \end{bmatrix}, \quad
\begin{bmatrix} 0 \\ 1 \\ 0 \\ \vdots \\ 0 \end{bmatrix}, \quad \cdots  \quad \text{or} \qquad
\begin{bmatrix} 0 \\ 0 \\ 0 \\ \vdots \\ 1 \end{bmatrix}.
$$

For example, if $x^{(i)}$ is an image of the digit 5, then the corresponding $y^{(i)}$ (that you should use with the cost function) should be a 10-dimensional vector with $y_5 = 1$, and the other elements equal to 0.

You should implement the feedforward computation that computes $h_w(x^{(i)})$ for every example $i$ and sum the cost over all examples. **Your code should also work for a dataset of any size, with any number of labels** (you can assume that there are always at least $K \ge 3$ labels).

### Regularized cost function

The cost function for neural networks with regularization is given by:


$$ J(w) = \frac{1}{m} \sum_{i=1}^{m}\sum_{k=1}^{K} \left[ - y_k^{(i)} \log \left( \left( h_w \left( x^{(i)} \right) \right)_k \right) - \left( 1 - y_k^{(i)} \right) \log \left( 1 - \left( h_w \left( x^{(i)} \right) \right)_k \right) \right] + \frac{\lambda}{2 m} \left[ \sum_{l=1}^{L-1} \sum_{i=1}^{S_{l}} \sum_{j=1}^{S_{l+1}} \left( w_{ji}^{(l)} \right)^2 \right] $$

Note that the terms corresponding to the bias should not be regularized. For the matrices `w1` and `w2`, this corresponds to the first column of each matrix. You should now add regularization to your cost function.

In [20]:
# Setup the parameters you will use for this exercise
input_layer_size  = 28*28   # 28x28 Input Images of Digits
hidden_layer_size =  25  # 25 hidden units
num_labels =    10   # 10 labels, from 0 to 9

In [21]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_=0.0):
    """
    Implements the neural network cost function for a two layer neural 
    network which performs classification.
    
    Parameters
    ----------
    nn_params : array_like
        The parameters for the neural network which are "unrolled" into 
        a vector.
    
    input_layer_size : int : Number of features for the input layer. 
    
    hidden_layer_size : int : Number of hidden units in the second layer.
    
    num_labels : int : Total number of labels, or equivalently number of units in output layer. 
    
    X : array_like : Input dataset. A matrix of shape (m x input_layer_size).
    
    y : array_like : Dataset labels. A vector of shape (m,).
    
    lambda_ : float, optional : Regularization parameter.
 
    Returns
    -------
    J : float : The computed value for the cost function at the current weight values.
    
    """
    # nn_params----> 25*401+10*26 unrolled into a row vector; s(j+1)*s(j)+1
    # Reshape nn_params back into the parameters w1 and w2, the weight matrices
    # for our 2 layer neural network
    w1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    w2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    # Setup some useful variables
    m = y.size
         
    # You need to return the following variables correctly 
    J = 0
    w1_grad = np.zeros(w1.shape)
    w2_grad = np.zeros(w2.shape)

    # ================================================================================================

    X = np.concatenate((np.ones((m, 1)), X), axis=1)
    z2 = X.dot(w1.T)
    a2 = sigmoid(z2)
    a2 = np.concatenate((np.ones((m, 1)), a2), axis=1)
    z3 = a2.dot(w2.T)
    a3 = sigmoid(z3)
    p = np.argmax(a3, axis=1)
    y = y.reshape(-1, 1)
    y = y == np.arange(num_labels)

    J = -1 / m * np.sum(y * np.log(a3) + (1 - y) * np.log(1 - a3)) + lambda_ / (2 * m) * (np.sum(w1[:, 1:] ** 2) + np.sum(w2[:, 1:] ** 2))


    

    
    # ===================================================================================================
  
    return J

## Backpropagation- A brief introduction.

Backpropagation is the heart of every neural network. Firstly, we need to make a distinction between backpropagation and optimizers.

Backpropagation is for calculating the gradients efficiently, while optimizers is for training the neural network, using the gradients computed with backpropagation. In short, all backpropagation does for us is compute the gradients. Nothing more.

### How does it work

We always start from the output layer and propagate backwards, updating weights and biases for each layer.

The idea is simple: adjust the weights and biases throughout the network, so that we get the desired output in the output layer. Say we wanted the output neuron to be 1.0, then we would need to nudge the weights and biases so that we get an output closer to 1.0.

We can only change the weights and biases, but activations are direct calculations of those weights and biases, which means we indirectly can adjust every part of the neural network, to get the desired output — except for the input layer, since that is the dataset that you input.

### Computing Gradients

Firstly, let's start by defining the relevant equations. Note that any indexing explained earlier is left out here, and we abstract to each layer instead of each weight, bias or activation:

\begin{equation}
z^{(L)}=w^{(L)} \times x +b \newline
a^{(L)}=
    g\left(
    \boldsymbol{z}^{(L)}
    \right) \newline
    C=(a^{(L)}-y)^2
\end{equation}


The **gradient** of this Cost Function C in the backpropagation algorithm, can be then be thought of as the change  in the cost function in relation to a specific weight. Therefore the change of Cost Function with respect to weights w can be best captured by:

\begin{equation}
\frac{\partial C}{\partial w^{(L)}}
    =
    \frac{\partial C}{\partial a^{(L)}}
    \frac{\partial a^{(L)}}{\partial z^{(L)}}
    \frac{\partial z^{(L)}}{\partial w^{(L)}}
    =
    2 \left(a^{(L)} - y \right) \sigma' \left(z^{(L)}\right) x^{(L-1)}
\end{equation}

Then you would update the weights after each mini-batch. Each weight is 'nudged' a certain amount for each layer l
\begin{equation}
w^{(l)} = w^{(l)} - \text{learning rate} \times \frac{\partial C}{\partial w^{(l)}}
\end{equation}

We have to move all the way back through the network and adjust each weight, iteratively.

## Backpropagation Method

In this part of the exercise, you will implement the backpropagation algorithm to compute the gradient for the neural network cost function. You will need to update the function `nnCostFunction` so that it returns an appropriate value for `grad`. Once you have computed the gradient, you will be able to train the neural network by minimizing the cost function $J(w)$ using an advanced optimizer such as `scipy`'s `optimize.minimize`.
You will first implement the backpropagation algorithm to compute the gradients for the parameters for the (unregularized) neural network. After you have verified that your gradient computation for the unregularized case is correct, you will implement the gradient for the regularized neural network.

<a id="section4"></a>
### Backpropagation

![](Figures/ex4-backpropagation.png)

Now, you will implement the backpropagation algorithm. Recall that the intuition behind the backpropagation algorithm is as follows. Given a training example $(x^{(t)}, y^{(t)})$, we will first run a “forward pass” to compute all the activations throughout the network, including the output value of the hypothesis $h_w(x)$. Then, for each node $j$ in layer $l$, we would like to compute an “error term” $\delta_j^{(l)}$ that measures how much that node was “responsible” for any errors in our output.

For an output node, we can directly measure the difference between the network’s activation and the true target value, and use that to define $\delta_j^{(3)}$ (since layer 3 is the output layer). For the hidden units, you will compute $\delta_j^{(l)}$ based on a weighted average of the error terms of the nodes in layer $(l+1)$. In detail, here is the backpropagation algorithm (also depicted in the figure above). You should implement steps 1 to 4 in a loop that processes one example at a time. Concretely, you should implement a for-loop `for t in range(m)` and place steps 1-4 below inside the for-loop, with the $t^{th}$ iteration performing the calculation on the $t^{th}$ training example $(x^{(t)}, y^{(t)})$. Step 5 will divide the accumulated gradients by $m$ to obtain the gradients for the neural network cost function.

1. Set the input layer’s values $(a^{(1)})$ to the $t^{th }$training example $x^{(t)}$. Perform a feedforward pass, computing the activations $(z^{(2)}, a^{(2)}, z^{(3)}, a^{(3)})$ for layers 2 and 3. Note that you need to add a `+1` term to ensure that the vectors of activations for layers $a^{(1)}$ and $a^{(2)}$ also include the bias unit. In `numpy`, if a 1 is a column matrix, adding one corresponds to `a_1 = np.concatenate([np.ones((m, 1)), a_1], axis=1)`.

1. For each output unit $k$ in layer 3 (the output layer), set 
$$\delta_k^{(3)} = \left(a_k^{(3)} - y_k \right)$$
where $y_k \in \{0, 1\}$ indicates whether the current training example belongs to class $k$ $(y_k = 1)$, or if it belongs to a different class $(y_k = 0)$. You may find logical arrays helpful for this task (explained in the previous programming exercise).

1. For the hidden layer $l = 2$, set 
$$ \delta^{(2)} = \left( w^{(2)} \right)^T \delta^{(3)} * g'\left(z^{(2)} \right)$$
Note that the symbol $*$ performs element wise multiplication in `numpy`.

1. Accumulate the gradient from this example using the following formula. Note that you should skip or remove $\delta_0^{(2)}$. In `numpy`, removing $\delta_0^{(2)}$ corresponds to `delta_2 = delta_2[1:]`.

1. Obtain the (unregularized) gradient for the neural network cost function by dividing the accumulated gradients by $\frac{1}{m}$:
$$ \frac{\partial}{\partial w_{ij}^{(l)}} J(w) = D_{ij}^{(l)} = \frac{1}{m} \Delta_{ij}^{(l)}$$

<a id="section3"></a>
### Sigmoid Gradient

To help you get started with this part of the exercise, you will first implement
the sigmoid gradient function. The gradient for the sigmoid function can be
computed as

$$ g'(z) = \frac{d}{dz} g(z) = g(z)\left(1-g(z)\right) $$

where

$$ \text{sigmoid}(z) = g(z) = \frac{1}{1 + e^{-z}} $$

Now complete the implementation of `sigmoidGradient` in the next cell.
<a id="sigmoidGradient"></a>

In [22]:
def sigmoidGradient(z):
    """
    Computes the gradient of the sigmoid function evaluated at z. 
    
    Parameters
    ----------
    z : array_like : A vector or matrix as input to the sigmoid function. 
    
    Returns
    --------
    g : array_like : Gradient of the sigmoid function. Has the same shape as z. 
    
    """

    # ============================================================

    g = sigmoid(z) * (1 - sigmoid(z))



    # =============================================================
    return g

### Random Initialization

When training neural networks, it is important to randomly initialize the parameters for symmetry breaking. One effective strategy for random initialization is to randomly select values for $w^{(l)}$ uniformly in the range $[-\epsilon_{init}, \epsilon_{init}]$. We are going to use $\epsilon_{init} = 0.12$. This range of values ensures that the parameters are kept small and makes the learning more efficient.

Use the function `randInitializeWeights` to initialize the weights for $w$. Note that the function has been provided with an argument for $\epsilon$ with default value `epsilon_init = 0.12`.

In [23]:
def randInitializeWeights(L_in, L_out, epsilon_init=0.12):
    """
    Randomly initialize the weights of a layer in a neural network.
    
    Parameters
    ----------
    L_in : int : Number of incomming connections.
    
    L_out : int : Number of outgoing connections. 
    
    epsilon_init : float, optional : Range of values which the weight can take from a uniform distribution.
    
    """
    W = np.zeros((L_out, 1 + L_in))
    W = np.random.rand(L_out, 1 + L_in) * 2 * epsilon_init - epsilon_init

    return W

In [24]:
#randomly initialise weights for two different layers
initial_w1 = randInitializeWeights(input_layer_size, hidden_layer_size)
initial_w2 = randInitializeWeights(hidden_layer_size, num_labels)

# Unroll parameters into a single array
initial_nn_params = np.concatenate([initial_w1.ravel(), initial_w2.ravel()], axis=0)

In [25]:
initial_w1

array([[ 0.01981368, -0.0408904 , -0.00471446, ...,  0.10997021,
         0.00587463, -0.07657741],
       [ 0.04998198,  0.06575506,  0.11017299, ...,  0.01644623,
        -0.08328651, -0.08002202],
       [-0.00353514, -0.07307795,  0.01792132, ..., -0.08233803,
        -0.07810803,  0.09933497],
       ...,
       [ 0.09944589, -0.03908655,  0.02977544, ..., -0.06523293,
        -0.10731024, -0.11541462],
       [-0.03504189,  0.06437448, -0.11501863, ...,  0.08878978,
         0.00587946,  0.04876764],
       [-0.0433984 ,  0.07158922,  0.05569794, ..., -0.09679383,
         0.07867625,  0.08769305]])

<a id="section5"></a>
### Regularized Neural Network

After you have successfully implemented the backpropagation algorithm, you also need to add regularization to the gradient. To account for regularization, it turns out that you can add this as an additional term *after* computing the gradients using backpropagation.

Specifically, after you have computed $\Delta_{ij}^{(l)}$ using backpropagation, you should add regularization using

$$ \begin{align} 
& \frac{\partial}{\partial w_{ij}^{(l)}} J(w) = D_{ij}^{(l)} = \frac{1}{m} \Delta_{ij}^{(l)} & \qquad \text{for } j = 0 \\
& \frac{\partial}{\partial w_{ij}^{(l)}} J(w) = D_{ij}^{(l)} = \frac{1}{m} \Delta_{ij}^{(l)} + \frac{\lambda}{m} w_{ij}^{(l)} & \qquad \text{for } j \ge 1
\end{align}
$$

Note that you should *not* be regularizing the first column of $w^{(l)}$ which is used for the bias term. Furthermore, in the parameters $w_{ij}^{(l)}$, $i$ is indexed starting from 1, and $j$ is indexed starting from 0. Thus, 

$$
w^{(l)} = \begin{bmatrix}
w_{1,0}^{(i)} & w_{1,1}^{(l)} & \cdots \\
w_{2,0}^{(i)} & w_{2,1}^{(l)} & \cdots \\
\vdots &  ~ & \ddots
\end{bmatrix}
$$

In [26]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_=0.0):
    """
    Implements the neural network cost function and gradient for a two layer neural 
    network which performs classification. 
    
    Parameters
    ----------
    nn_params : array_like
        The parameters for the neural network which are "unrolled" into 
        a vector.
    
    input_layer_size : int
        Number of features for the input layer. 
    
    hidden_layer_size : int
        Number of hidden units in the second layer.
    
    num_labels : int
        Total number of labels, or equivalently number of units in output layer. 
    
    X : array_like
        Input dataset. A matrix of shape (m x input_layer_size).
    
    y : array_like
        Dataset labels. A vector of shape (m,).
    
    lambda_ : float, optional
        Regularization parameter.
 
    Returns
    -------
    J : float : The computed value for the cost function at the current weight values.
    
    grad : array_like : An "unrolled" vector of the partial derivatives of the concatenatation of
                        neural network weights w1 and w2.
    """
    
    # Reshape nn_params back into the parameters w1 and w2, the weight matrices
    # for our 2 layer neural network
    w1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    w2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    # Setup some useful variables
    m = y.size
         
    # You need to return the following variables correctly 
    J = 0
    w1_grad = np.zeros(w1.shape)
    w2_grad = np.zeros(w2.shape)

    # ================================================================================================

    X = np.concatenate((np.ones((m, 1)), X), axis=1)
    z2 = X.dot(w1.T)
    a2 = sigmoid(z2)
    a2 = np.concatenate((np.ones((m, 1)), a2), axis=1)
    z3 = a2.dot(w2.T)
    a3 = sigmoid(z3)
    p = np.argmax(a3, axis=1)
    y = y.reshape(-1, 1)
    y = y == np.arange(num_labels)

    J = -1 / m * np.sum(y * np.log(a3) + (1 - y) * np.log(1 - a3)) + lambda_ / (2 * m) * (np.sum(w1[:, 1:] ** 2) + np.sum(w2[:, 1:] ** 2))

    # Backpropogation
    d3 = a3 - y
    d2 = d3.dot(w2[:, 1:]) * sigmoidGradient(z2)
    w1_grad = 1 / m * d2.T.dot(X)
    w2_grad = 1 / m * d3.T.dot(a2)

    # Add regularization to gradient
    w1_grad[:, 1:] = w1_grad[:, 1:] + lambda_ / m * w1[:, 1:]
    w2_grad[:, 1:] = w2_grad[:, 1:] + lambda_ / m * w2[:, 1:]

    #=======================================================================================================

    # Unroll gradients
    grad = np.concatenate([w1_grad.ravel(), w2_grad.ravel()])
    return J, grad

### Learning parameters using `scipy.optimize.minimize`

After you have successfully implemented the neural network cost function
and gradient computation, the next step we will use `scipy`'s minimization to learn a good set parameters.

In [27]:
#  After you have completed the assignment, change the maxiter to a larger value to see how more training helps.
options= {'maxiter': 100}

#  You should also try different values of lambda
lambda_ = 0.1

# Create "short hand" for the cost function to be minimized
costFunction = lambda p: nnCostFunction(p, input_layer_size, # p == nn_param
                                        hidden_layer_size,
                                        num_labels, X, y, lambda_)

# Now, costFunction is a function that takes in only one argument
# (the neural network parameters)
res = optimize.minimize(costFunction,
                        initial_nn_params,
                        jac=True,
                        method='TNC',
                        options=options)



<ipython-input-13-c622fa87c6ed>:36: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-z))


KeyboardInterrupt: 

In [84]:
res

     fun: 0.6933524165748763
     jac: array([ 1.68322852e-03, -2.04567051e-05,  2.72947999e-05, ...,
       -3.93194162e-03,  1.14572119e-04, -2.89250002e-03])
 message: 'Max. number of function evaluations reached'
    nfev: 101
     nit: 12
  status: 3
 success: False
       x: array([ 0.45554158, -0.03409451,  0.04549133, ...,  1.15501903,
       -0.29830072,  1.07079938])

In [85]:
# get the solution of the optimization
nn_params = res.x #Row vector containing weight values for every layer of your implemented neural network
        
# Obtain w1 and w2 back from nn_params


After the training completes, we will proceed to report the training accuracy of your classifier by computing the percentage of examples it got correct. If your implementation is correct, you should see a reported
training accuracy of about 95.3% (this may vary by about 1% due to the random initialization). It is possible to get higher training accuracies by training the neural network for more iterations. We encourage you to try
training the neural network for more iterations (e.g., set `maxiter` to 400) and also vary the regularization parameter $\lambda$. With the right learning settings, it is possible to get the neural network to perfectly fit the training set.

In [86]:
pred = predict(w1, w2, X)
print('Training Set Accuracy: %f' % (np.mean(pred == y) * 100))

Training Set Accuracy: 94.760000


In [ ]:
if indices.size > 0:
    i, indices = indices[0], indices[1:]
    displayData(X[i, :], figsize=(4, 4))
    pred = predict(w1, w2, X[i, :])
    print('Neural Network Prediction: {}'.format(*pred))
else:
    print('No more images to display!')

## Supplementary: One Hot Vector Encoding

https://www.reddit.com/r/MachineLearning/comments/31fk7i/converting_target_indices_to_onehotvector/
https://stackoverflow.com/questions/45068853/how-does-this-one-hot-vector-conversion-work

In [ ]:
optimize.minimize?